In [1]:
import itertools
import random
from collections import defaultdict
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
test_path = '/content/drive/My Drive/circuits/c432.test'

In [4]:
import itertools
import random
from collections import defaultdict
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def expand_dont_care_limited(pattern, max_expand=15):
    x_indices = [i for i, c in enumerate(pattern) if c == 'x']
    if len(x_indices) <= max_expand:
        chars = [(c if c in '01' else ['0', '1']) for c in pattern]
        chars = [(c if isinstance(c, list) else [c]) for c in chars]
        return [''.join(bits) for bits in itertools.product(*chars)]
    else:
        fixed_pattern = list(pattern)
        patterns = []
        for bits in itertools.product('01', repeat=max_expand):
            temp = fixed_pattern.copy()
            for i, bit in zip(x_indices[:max_expand], bits):
                temp[i] = bit
            for i in x_indices[max_expand:]:
                temp[i] = random.choice('01')
            patterns.append(''.join(temp))
        return patterns

def parse_isc_file_limited(filepath, max_expand=3):
    combo_to_fault = {}
    fault_to_idx = {}
    fault_type_map = {}
    idx = 0
    current_fault = None

    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("*"):
                continue
            if '/' in line:
                parts = line.split("/")
                fault = parts[0].strip().replace("->", "_") + "/" + parts[1].strip()
                if fault not in fault_to_idx:
                    fault_to_idx[fault] = idx
                    if parts[1].strip() == "0":
                        fault_type_map[idx] = "stuck-at-0"
                    elif parts[1].strip() == "1":
                        fault_type_map[idx] = "stuck-at-1"
                    else:
                        fault_type_map[idx] = "unknown"
                    idx += 1
                current_fault = fault
            elif ':' in line and current_fault:
                parts = line.split(":")[1].strip().split()
                input_pat = parts[0]
                output_pat = parts[1] if len(parts) > 1 else ""
                for xi in expand_dont_care_limited(input_pat, max_expand):
                    for yo in expand_dont_care_limited(output_pat, max_expand):
                        combined = xi + yo
                        if combined not in combo_to_fault:
                            combo_to_fault[combined] = fault_to_idx[current_fault]
    return combo_to_fault, fault_to_idx, fault_type_map

In [5]:
class FaultDatasetMC(Dataset):
    def __init__(self, combo_to_fault):
        self.samples = []
        for combo, label in combo_to_fault.items():
            features = [int(c) for c in combo]
            self.samples.append((features, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return (
            torch.tensor(x, dtype=torch.float32).squeeze().numpy(),  # Flatten input for Logistic Regression
            y
        )

In [6]:
# Load data
combo_to_fault, fault_to_idx, fault_type_map = parse_isc_file_limited(test_path, max_expand=3)
dataset = FaultDatasetMC(combo_to_fault)

# Split data
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(train_set, batch_size=len(train_set))
val_loader = DataLoader(val_set, batch_size=len(val_set))

# Convert data loaders to NumPy arrays
def convert_loader_to_numpy(loader):
    X_list = []
    y_list = []
    for X, y in loader:
        X_list.append(X)
        y_list.append(y.numpy())
    if X_list:
        X = np.concatenate(X_list, axis=0)
        y = np.concatenate(y_list, axis=0)
        return X, y
    else:
        return np.array([]), np.array([])

X_train, y_train = convert_loader_to_numpy(train_loader)
X_val, y_val = convert_loader_to_numpy(val_loader)

In [7]:
# Train LASSO Logistic Regression
lasso_model = LogisticRegression(
    penalty='l1',
    solver='saga',
    C=1.0,
    max_iter=100,
    random_state=SEED,
    multi_class='ovr'
)
lasso_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(multi_class='ovr', penalty='l1', random_state=42,
                   solver='saga')

In [8]:
# Evaluate
y_pred = lasso_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")

# Evaluate stuck-at fault accuracy
type_correct = defaultdict(int)
type_total = defaultdict(int)

for true_label, pred_label in zip(y_val, y_pred):
    true_type = fault_type_map.get(true_label, "unknown")
    type_total[true_type] += 1
    if true_label == pred_label:
        type_correct[true_type] += 1

stuck0_acc = type_correct['stuck-at-0'] / type_total['stuck-at-0'] if type_total['stuck-at-0'] > 0 else 0
stuck1_acc = type_correct['stuck-at-1'] / type_total['stuck-at-1'] if type_total['stuck-at-1'] > 0 else 0

print(f"\n--- Final Stuck-at Fault Accuracy ---")
print(f"Stuck-at-0 Accuracy: {stuck0_acc:.4f} ({type_correct['stuck-at-0']}/{type_total['stuck-at-0']})")
print(f"Stuck-at-1 Accuracy: {stuck1_acc:.4f} ({type_correct['stuck-at-1']}/{type_total['stuck-at-1']})")

Validation Accuracy: 0.7350
Validation F1 Score: 0.5938

--- Final Stuck-at Fault Accuracy ---
Stuck-at-0 Accuracy: 0.6729 (1325/1969)
Stuck-at-1 Accuracy: 0.7646 (3145/4113)
